### Training with deeper CNN

In [21]:
import numpy as np
import torch as th
import torch.optim as optim

from torch import nn
from torch.utils.data import DataLoader
from cnn import CNN
from tqdm import tqdm
from imgdata import DefaultTrainSet, DefaultTestSet

In [22]:
%matplotlib inline

if th.cuda.is_available():
  # Make CuDNN Determinist
  th.backends.cudnn.deterministic = True

# Define default device, we should use the GPU (cuda) if available
device = th.device("cuda" if th.cuda.is_available() else "cpu")
device

device(type='cpu')

### Set constants

In [23]:
lr = 1e-4
epochs = 25
batch_size = 16
class_label = ['faces', 'dog', 'airplanes', 'keyboard', 'cars']

In [24]:
train_set = DefaultTrainSet()

train_dataloader = th.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)

### Define model

In [25]:
cnn = CNN()
fc_inputs = 64 * 30 * 30

In [26]:
cnn.fc = nn.Sequential(
    nn.Linear(fc_inputs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 5)
)

In [27]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=lr)

### Train model

In [28]:
def train(_net, n_epochs, train_loader, _optimizer, _criterion):
    """
    Train a neural network and print statistics of the training

    :param _criterion:
    :param _optimizer:
    :param train_loader: loader for the training data
    :param _net: (PyTorch Neural Network)
    :param n_epochs: (int)  Number of iterations on the training set
    """
    print("---------- HYPERPARAMETERS ----------")
    print("n_epochs=", n_epochs)
    print("-" * 40)
    accuracies = []
    losses = []
    # Move model to gpu if possible
    _net = _net.to(device)
    for epoch_count, epoch in enumerate(range(n_epochs)):
        # Set the model to training mode
        _net.train()
        # Initialize the variables to compute the average loss
        total_loss = 0.00
        total_correct = 0
        pbar = tqdm(train_loader)
        # Iterate over the training set
        for i, data in enumerate(pbar):

            # Get the inputs
            inputs, labels = data['imNorm'], data['label']
            # Move them to the correct device
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the parameter gradients
            _optimizer.zero_grad()

            # Forward pass
            outputs = _net(inputs)
             # Calculate Loss: softmax --> cross entropy loss
            loss = _criterion(outputs, labels)
            _, preds = th.max(outputs, 1)
            # Getting gradients w.r.t. parameters
            loss.backward()

            # Updating parameters
            _optimizer.step()

            # Update the running variables
            total_loss += loss.item()
            avg_loss = total_loss / (i + 1)
            total_correct += th.sum(preds == labels.data)
            pbar.set_description("Epoch: {}/{}".format(epoch_count + 1, n_epochs))
            pbar.set_postfix(avergae_loss=avg_loss, running_accuracy=th.sum(preds == labels).item() / float(labels.size(0)))
            # save the model
            model_name = f"Outputs/models/deeperCNN.pth"
            th.save(_net.state_dict(), model_name)

        # Calculate the average loss and accuracy of the epoch
        losses.append(total_loss/len(train_loader))
        accuracies.append((total_correct/train_loader.sampler.num_samples).item())
        # print losses and accuracies in the last epoch
        if epoch_count == n_epochs - 1:
            print("---------- FINAL RESULTS ----------")
            print("Losses fot every epoch:", losses)
            print("Average accuracy for each epoch: {}".format(accuracies))
            print("-" * 40)
    return losses, accuracies

In [29]:
losses, accuracies = train(cnn, epochs, train_dataloader, optimizer, loss_func)

---------- HYPERPARAMETERS ----------
n_epochs= 25
----------------------------------------


Epoch: 25/25: 100%|██████████| 22/22 [00:13<00:00,  1.65it/s, avergae_loss=0.0113, running_accuracy=1]   

---------- FINAL RESULTS ----------
Losses fot every epoch: [1.9271153861826116, 0.6749441894617948, 0.49776759269562637, 0.3656607930633155, 0.2724817950617183, 0.2123952717943625, 0.13434427146884528, 0.0995273338207467, 0.08660318135199221, 0.07446989311244, 0.05020524317991327, 0.059764871881766754, 0.07643645352006635, 0.05262225438756021, 0.04026477105534551, 0.02323203709569167, 0.02657177110879936, 0.01839774085039442, 0.02060456777160818, 0.018285252032166518, 0.0148158241645433, 0.01375924391058189, 0.009718566609080881, 0.0071413194951178, 0.01134635950174098]
Average accuracy for each epoch: [0.49714285135269165, 0.7571428418159485, 0.8199999928474426, 0.8742856979370117, 0.9057142734527588, 0.9142857193946838, 0.9657142758369446, 0.9685714244842529, 0.9800000190734863, 0.977142870426178, 0.9885714054107666, 0.9857142567634583, 0.9828571677207947, 0.9942857027053833, 0.9885714054107666, 1.0, 0.9885714054107666, 1.0, 0.9942857027053833, 0.9971428513526917, 0.9971428513526917

### Evaluate model

In [30]:
# load test set and create dataloader
test_set = DefaultTestSet()
test_loader = DataLoader(test_set)
test_y = [i['label'] for i in test_set]

In [31]:
def predict(_model, _tests):
  predicts = []
  with th.no_grad():
    for i in tqdm(_tests):
        _model.eval()
        images = i['imNorm'].clone().to(device)
        outputs = _model(images)
        _, predicted = th.max(outputs.data, 1)
        predicts.append(predicted.item())
  return predicts

cnn.load_state_dict(th.load('Outputs/models/deeperCNN.pth'))
predict_y = predict(cnn, test_loader)

100%|██████████| 50/50 [00:00<00:00, 54.44it/s]


In [32]:
correct = []
incorrect = []
for i, j in enumerate(predict_y):
    if j == test_y[i]:
      correct.append(i)
    else:
      incorrect.append(i)

# generate confusion matrix
def confusion_matrix(y_true, y_pred):
  n_class = np.unique(y_true).shape[0]
  cm = np.zeros((n_class, n_class))
  for i in range(len(y_true)):
    cm[y_true[i], y_pred[i]] += 1
  return cm


cm = confusion_matrix(test_y, predict_y)
print(cm)

def print_accuracy(cm):
  for i in range(5):
    print(f'the accuracy by class {class_label[i]} : {cm[i, i] / np.sum(cm[i, :])}')
  # overall accuracy
  print(f'overall accuracy : {np.sum(cm[range(5), range(5)]) / np.sum(cm)}')

print_accuracy(cm)

[[ 6.  2.  0.  1.  1.]
 [ 0.  8.  0.  2.  0.]
 [ 0.  3.  7.  0.  0.]
 [ 0.  4.  0.  5.  1.]
 [ 0.  0.  0.  0. 10.]]
the accuracy by class faces : 0.6
the accuracy by class dog : 0.8
the accuracy by class airplanes : 0.7
the accuracy by class keyboard : 0.5
the accuracy by class cars : 1.0
overall accuracy : 0.72
